# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,klyuchi,52.2667,79.1667,7.13,45,94,4.71,RU,1727412235
1,1,papatowai,-46.5619,169.4708,6.83,86,81,5.85,NZ,1727412236
2,2,port-aux-francais,-49.3500,70.2167,3.56,74,100,14.91,TF,1727412238
3,3,ocean shores,46.9737,-124.1563,13.34,96,100,9.26,US,1727412239
4,4,dwarka,22.2394,68.9678,28.17,78,100,7.05,IN,1727412240


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size = "Humidity"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] < 10) & (city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 20) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Humidity"] < 30)]

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,khash,28.2211,61.2158,27.47,19,0,2.56,IR,1727412264
164,164,kapiri mposhi,-13.9652,28.6809,20.14,26,1,2.69,ZM,1727412522
196,196,port augusta,-32.5000,137.7667,25.28,14,2,6.41,AU,1727412572
296,296,kharan,28.5833,65.4167,29.41,25,1,4.20,PK,1727412716
318,318,balkanabat,39.5108,54.3671,21.69,19,0,5.27,TM,1727412747
424,424,ivins,37.1686,-113.6794,27.75,18,0,1.79,US,1727412907
471,471,al hasakah,36.5024,40.7477,26.74,23,0,4.46,SY,1727412974
495,495,sao vicente,-23.9631,-46.3919,21.10,26,2,5.20,BR,1727412884
507,507,pitangueiras,-21.0094,-48.2217,25.23,29,4,1.05,BR,1727413025


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,khash,IR,28.2211,61.2158,19,
164,kapiri mposhi,ZM,-13.9652,28.6809,26,
196,port augusta,AU,-32.5000,137.7667,14,
296,kharan,PK,28.5833,65.4167,25,
318,balkanabat,TM,39.5108,54.3671,19,
424,ivins,US,37.1686,-113.6794,18,
471,al hasakah,SY,36.5024,40.7477,23,
495,sao vicente,BR,-23.9631,-46.3919,26,
507,pitangueiras,BR,-21.0094,-48.2217,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
khash - nearest hotel: No hotel found
kapiri mposhi - nearest hotel: Chanika Lodge
port augusta - nearest hotel: Hotel Commonwealth
kharan - nearest hotel: No hotel found
balkanabat - nearest hotel: Nebitçi
ivins - nearest hotel: Crescent Moon Inn
al hasakah - nearest hotel: فندق السنابل
sao vicente - nearest hotel: Pousada Vitória
pitangueiras - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
20,khash,IR,28.2211,61.2158,19,No hotel found
164,kapiri mposhi,ZM,-13.9652,28.6809,26,Chanika Lodge
196,port augusta,AU,-32.5000,137.7667,14,Hotel Commonwealth
296,kharan,PK,28.5833,65.4167,25,No hotel found
318,balkanabat,TM,39.5108,54.3671,19,Nebitçi
424,ivins,US,37.1686,-113.6794,18,Crescent Moon Inn
471,al hasakah,SY,36.5024,40.7477,23,فندق السنابل
495,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória
507,pitangueiras,BR,-21.0094,-48.2217,29,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",  
    size = 60,  
    hover_cols = ["Lng", "Lat", "Humidity", "City", "Country", "Hotel Name"],
    xlabel = "Longitude",  
    ylabel = "Latitude",  
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)